In [ ]:
import glob
import os
import librosa
import numpy as np
import seaborn as sns
import wave
import contextlib
import pandas as pd

from sklearn.metrics import accuracy_score,classification_report,f1_score
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
%matplotlib inline
plt.style.use('ggplot')

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 11
plt.rcParams['figure.titlesize'] = 13

 # Features plots - Code

In [ ]:
def load_sound_files(file_paths):
    raw_sounds = []
    for fp in file_paths:
        X,sr = librosa.load(fp)
        raw_sounds.append(X)
    return raw_sounds

def plot_waves(sound_names,raw_sounds):
    i = 1
    fig = plt.figure(figsize=(25,60), dpi = 900)
    for n,f in zip(sound_names,raw_sounds):
        plt.subplot(10,1,i)
        librosa.display.waveplot(np.array(f),sr=22050)
        plt.title(n.title())
        i += 1
    plt.suptitle('Figure 1: Waveplot',x=0.5, y=0.915,fontsize=18)
    plt.show()
    
def plot_specgram(sound_names,raw_sounds):
    i = 1
    fig = plt.figure(figsize=(25,60), dpi = 900)
    for n,f in zip(sound_names,raw_sounds):
        plt.subplot(10,1,i)
        specgram(np.array(f), Fs=22050)
        plt.title(n.title())
        i += 1
    plt.suptitle('Figure 2: Spectrogram',x=0.5, y=0.915,fontsize=18)
    plt.show()

def plot_log_power_specgram(sound_names,raw_sounds):
    i = 1
    fig = plt.figure(figsize=(25,60), dpi = 900)
    for n,f in zip(sound_names,raw_sounds):
        plt.subplot(10,1,i)
        D = librosa.logamplitude(np.abs(librosa.stft(f))**2, ref_power=np.max)
        librosa.display.specshow(D,x_axis='time' ,y_axis='log')
        plt.title(n.title())
        i += 1
    plt.suptitle('Figure 3: Log power spectrogram',x=0.5, y=0.915,fontsize=18)
    plt.show()

# Extracting features

In [ ]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    #print(chroma.shape)
    #print("chroma")
    #print(mfccs.shape)
    #print("mfccs")
    #print(mel.shape)
    #print("mels")
    #print(contrast.shape)
    #print("contrast")
    #print(tonnetz.shape)
    #print("tonnetz")    
    return mfccs,chroma,mel,contrast,tonnetz

def parse_audio_files(parent_dir,sub_dirs,file_ext='*.wav'):
    d=0
    duration1 = []
    duration2 = []
    for label, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir,sub_dir,file_ext)):
            with contextlib.closing(wave.open(fn,'r')) as f:
                frames = f.getnframes()
                rate = f.getframerate()
                duration = frames / float(rate)
                print(pd.Series(fn.split('_')).unique())
                print(d,duration)
                d=d+1
                print(" ")
                #dn=0
                if(sub_dir!='noise_set'):
                    #if(duration<0):
                    #    dn =dn+1
                    duration1.append(duration)  
                else:
                    duration2.append(duration)
            
    return duration1,duration2

In [ ]:
def parse_audio_files2(parent_dir,sub_dirs,file_ext='*.wav'):
    features = np.empty((0,193))
    finlabel = np.empty((0,3))
    labells=[]
    labells1=np.empty(0) 
    labells2=np.empty(0) 
    d=0
    fns = []
    labels=[]
    label1s=[]
    label2s=[]
    for label, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir,sub_dir,file_ext)):
            mfccs, chroma, mel, contrast,tonnetz = extract_feature(fn)
            ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
          #  print("ext_features")
          #  print(ext_features.shape)
            features = np.vstack([features,ext_features])
          #  print("features")
          #  print(features.shape)
            if(sub_dir!='noise_set'):
                    d = d+1
                    if (len(fn.split('_'))>=6):
                        label1 = "1"
                    else:
                        label1="0"
                    label="1"
                    if(sub_dir=='TRIM_old'):
                        if(fn.split('_')[-1]=='2.wav' ):
                            label2="2"
                        if(fn.split('_')[3]=='Multiplepeacock'):
                            label2 = "4"
                        else:
                            label2 ="1"
                    else:
                        if(fn.split('_')[-1]=='1.wav'):
                            label2="1"
                        elif(fn.split('_')[-1]=='2.wav'):
                            label2 ="2" 
                        elif(fn.split('_')[-1]=='3.wav'):
                            label2 ="3"
                        elif(fn.split('_')[-1]=='MI.wav' or fn.split('_')[-1]=='3or4.wav'):
                            label2 ="4"                        
                        else:
                            label2 ="0"
                            
            else:
                label ="0"
                label1="1"
                label2 ="0"
            fns.append(fn.split('_')[1:])
            use = np.hstack([label,label1,label2])
            finlabel = np.vstack([finlabel,use])
            labels.append(label)
            label1s.append(label1)
            label2s.append(label2)
    return(fns, np.array(features), np.array(finlabel),np.array(labels), np.array(label1s), np.array(label2s))

> #  Feed Train & Test Data 

In [ ]:
parent_dir = '../input/data-w/data/data'

sub_dirs = ['C_new2','TRIM_old','A_new1','trimmed_new3','noise_set','Chincholi','iid','Kaan'] #train dirs
#sub_dirs2=['trimmed_new3','audio']        #test dirs

#features, labels,la1 , la2,la3 = 
duration1,duration2= parse_audio_files(parent_dir,sub_dirs)
#fea2,la2 = parse_audio_files(parent_dir,sub_dirs2)
#print(features)
#print(features.shape)
print('hiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii')
#print(labels)
#print(fea2)
#print(la2)

In [ ]:
dafr = pd.DataFrame()
dafr["peacock_audio_durations"] = pd.Series(duration1)
dafr["noise_durations"] = pd.Series(duration2)
#dafr["index"] = pd.Series(np.arange(0,78))

In [ ]:
sns.set(style="whitegrid")
ax = sns.violinplot(data=dafr)

In [ ]:
fn,features,final_labels,label,label1,label2 = parse_audio_files2(parent_dir,sub_dirs)

In [ ]:
final_labels.shape

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer,LabelBinarizer
from numpy import argmax
from keras.utils import to_categorical
from keras import initializers

one_hot1 = LabelBinarizer()
# One-hot encode data
label_ = one_hot1.fit_transform(label)
label_1 = one_hot1.fit_transform(label1)

print(one_hot1.classes_)
print(label_)
print(label_1)

one_hot2 = MultiLabelBinarizer()
# One-hot encode data
label_2 = one_hot2.fit_transform(label2)
label__2 = to_categorical(label2)
print(label__2)
one_hot2.classes_

In [ ]:
print(label__2.shape)
print(label_1.shape)
print(label_.shape)
print(label_2.shape)

In [ ]:
dafr_labels = pd.DataFrame()
dafr_labels["file_name"] = fn
dafr_labels["peacock"] = label_
dafr_labels["disturbance"] = label_1
#dafr_labels["single_peacock"] = [label__2]

In [ ]:
dafr_labels['peacock'].value_counts().plot(kind='bar')
            #,'disturbance','single_peacock']].plot(kind='bar')
#plt.xticks(np.arange(0, 51, 5)) 
plt.yticks(np.arange(0, 1200, 50)) 
plt.show()
dafr_labels['disturbance'].value_counts().plot(kind='bar')
plt.yticks(np.arange(0, 1200, 50)) 
plt.show()
#dafr_labels['single_peacock'].value_counts().plot(kind='bar')
#plt.yticks(np.arange(0, 1200, 25)) 
#plt.show()

In [ ]:
pd.Series(label).value_counts()

In [ ]:
pd.Series(label1).value_counts()

In [ ]:
pd.Series(label2).value_counts()

In [ ]:
#dafr_labels

# MLP keras 

In [ ]:
import keras 
from keras.models import Sequential
from keras.layers import Dense, Activation,Flatten, Dropout, BatchNormalization

from sklearn.preprocessing import StandardScaler
#from keras import metrics
from sklearn.model_selection import train_test_split
#from sklearn.metrics import f1_score
#from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D
#from keras import regularizers, optimizer
import pandas as pd
import random
import tensorflow as tf

from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
#from sklearn.preprocessing import LabelEncoder
#from sklearn.pipeline import Pipeline
from keras import backend as K


random.seed(42)
np.random.seed(42)
tf.set_random_seed(42)

my_init = initializers.glorot_uniform(seed=42)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [ ]:
#from keras.layers.core import Flatten
X_train, X_test, y1_train, y1_test,y2_train,y2_test,y3_train,y3_test = train_test_split(features,
                                            label_,label_1,label__2, test_size=0.33, random_state=42)
#train_test_split(X, Y1, Y2, Y3, random_state = 42)

In [ ]:
print(y3_train.shape)
print(y3_test.shape)

In [ ]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

## Multi output neural network

In [ ]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
#create the model
inn = Input(shape = (193,))
x = Sequential()(inn)
    # Adding the input layer and the first hidden layer
#x =
x = Dense(output_dim = 40, init = 'uniform', activation = 'relu', input_dim = 193)(x)
    # Adding the second hidden layer
#x = 
x=Dense(output_dim = 20, init = 'uniform', activation = 'relu')(x)
#x =
x= Dense(output_dim = 8, init = 'uniform', activation = 'relu')(x)
    # Adding the output layer
out1 =Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid')(x)
out2 =Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid')(x)
out3 =Dense(output_dim = 5, init = 'uniform', activation = 'softmax')(x)
    # Compiling Neural Network
model = Model(inputs=[inn], outputs=[out1, out2,out3])
#classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# compile the model
model.compile(optimizer='adam', loss=['binary_crossentropy','binary_crossentropy','categorical_crossentropy'],
              metrics=[f1_m,f1_m,f1_m])

In [ ]:
# fit the model
history = model.fit(X_train, [y1_train,y2_train,y3_train],
                    validation_split=0.3, epochs=100,batch_size = 10, verbose=1)

In [ ]:
# evaluate the model
f1score = model.evaluate(X_test, [y1_test,y2_test,y3_test])
#classifier =KerasClassifier(build_fn=making_model(193), epochs=100, batch_size=10, verbose=0)

In [ ]:
f1score

In [ ]:
from sklearn.metrics import confusion_matrix,precision_score
,recall_score,f1_score,cohen_kappa_score,multilabel_confusion_matrix
multilabel_confusion_matrix(X_test, [y1_test,y2_test,y3_test])

In [ ]:
train_test_split_indices = np.random.rand(len(features))<0.67

In [ ]:
pd.Series(train_test_split_indices).value_counts()

In [ ]:
#train and test data
X_train = features[train_test_split]
X_test  = features[~train_test_split]
y_train = [label_[train_test_split],label_1[train_test_split],label__2[train_test_split]]
y_test  = [label_[~train_test_split],label_1[~train_test_split],label__2[~train_test_split]]

In [ ]:
print(X_train.shape[1])

In [ ]:
visible = Input(shape=input_shape)
x1 = Conv2D(10, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(visible)
x1 = MaxPooling2D(pool_size=(2, 2))(x1)
x2 = Conv2D(20, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(x1)
x2 = MaxPooling2D(pool_size=(2, 2))(x2)
x3 = Conv2D(30, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(x2)
x = GlobalMaxPooling2D()(x3)
output = Dense(Y_train.shape[1], activation='softmax', name = 'emotion', kernel_initializer=my_init)(x)
output2 = Dense(Y_train2.shape[1], activation='sigmoid', kernel_initializer=my_init)(GlobalMaxPooling2D()(x2))
output3 = Dense(Y_train3.shape[1], activation='linear', kernel_initializer=my_init)(GlobalMaxPooling2D()(x1))
model = Model(inputs=visible, outputs=[output, output2, output3])

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(features, [label_,label_1,label__2], test_size = 0.33)
#print(X_train.shape)
#print(X_test.shape)

In [ ]:
label_ =  np.asarray(label_)
print(label_.shape)
label_1 =  np.asarray(label_1)
print(label_1.shape)
label__2 =  np.asarray(label__2)
print(label__2.shape)
l = [label_,label_1,label__2]
np.asarray(l).shape

In [ ]:
model = making_model(193)

In [ ]:
np.vstack(label_,label_1,label__2).shape

In [ ]:
from imblearn.keras import BalancedBatchGenerator
from sklearn.metrics import roc_auc_score

def fit_predict_balanced_model(X_train, y_train, X_test, y_test):
    model = classifier
    training_generator = BalancedBatchGenerator(X_train, y_train)
    model.fit_generator(generator=training_generator,epochs=100,verbose=1)
    y_pred = model.predict_proba(X_test)
    print(classification_report(y_test, y_pred))
    return roc_auc_score(y_test, y_pred)
res = fit_predict_balanced_model(X_train, y_train, X_test, y_test)

In [ ]:
classifier

In [ ]:
model

In [ ]:
# Fitting our model 
import eli5
from eli5.sklearn import PermutationImportance

history=model.fit(X_train, y_train,validation_data=(X_test, y_test),nb_epoch = 100)
#history=classifier.fit(X,y,validation_data=(X_test, y_test), batch_size = 10, nb_epoch = 100)
#acc = []
# Predicting the Test set results

In [ ]:
y_pred[:,0].shape

In [ ]:
y_pred = model.predict(X_test)
#print(pd.Series(y_pred[:,0]).value_counts())
#print(pd.Series(y_pred[:,1]).value_counts())
#print(pd.Series(y_pred[:,2]).value_counts())

In [ ]:
#y_pred = (y_pred > 0.5)
print('hiiiiii')
#print(y_test)
#print(y_pred)
err1=0
corr1=0
corr2=0
corr3=0
err2=0
err3=0
for i in range(len(y_test)):
    for j in range(3):
        if(y_pred[i][j]>=1.5 and y_pred[i][j]< 2.5):
            y_pred[i][j]=2
        elif(y_pred[i][j]>0.5 and y_pred[i][j]< 1.5):
            y_pred[i][j]=1
        elif(y_pred[i][j]>=2.5 and y_pred[i][j]< 3.5):
            y_pred[i][j]=3
        elif(y_pred[i][j]>=3.5 and y_pred[i][j]< 4.5):
            y_pred[i][j]=4
        else:
            y_pred[i][j]=0
for i in range(len(y_test)):            
    if(y_test[i][0]!=y_pred[i][0]):
        err1 = err1+1
        print(y_test[i][0],y_pred[i][0])
    else:
        corr1=corr1+1
print("______________________________________________________")
for i in range(len(y_test)):
    if(y_test[i][1]!=y_pred[i][1]):
        err2 = err2+1
        print(y_test[i][1],y_pred[i][1])
    else:
        corr2=corr2+1
print("______________________________________________________")
for i in range(len(y_test)):
    if(y_test[i][2]!=y_pred[i][2]):
        err3 = err3+1
        print(y_test[i][2],y_pred[i][2])
    else:
        corr3=corr3+1
print("______________________________________________________")
print('error')
print(err1)
print(corr1)
print(err2)
print(corr2)
print(err3)
print(corr3)
scores = model.evaluate(x=X_test, y=y_test, batch_size=None, verbose=2, sample_weight=None, steps=None)
print('Test loss:', scores)
#acc.append(accuracy)
# evaluate the model
#scores = model.evaluate(X[test], Y[test], verbose=0)
#print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
#cvscores.append(scores[1] * 100)
#print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

In [ ]:
f1_score(y_test[0],y_pred[0])

In [ ]:
f1_score(y_test[1],y_pred[1])

In [ ]:
f1_score(y_test[2],y_pred[2])

In [ ]:
#perm = PermutationImportance(model,scoring='accuracy', random_state=1).fit(X_train,y_train[0])
#eli5.show_weights(perm, feature_names = X_train.columns.tolist())

In [ ]:
plot_history(history)

In [ ]:
# Creating the Confusion Matrix

#y_test = np.transpose()
cm = confusion_matrix(y_test[:,0], y_pred[:,0])
print(cm)

In [ ]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,cohen_kappa_score
#y_test = np.transpose()
cm = confusion_matrix(y_test[:,1], y_pred[:,1])
print(cm)

In [ ]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,cohen_kappa_score
#y_test = np.transpose()
cm = confusion_matrix(y_test[:,2], y_pred[:,2])
print(cm)

In [ ]:
f1_score(y_test[:,0],y_pred[:,0])

In [ ]:
# F1 score
f1_score(y_test[:,1],y_pred[:,1])

In [ ]:
f1_score(y_test[:,2],y_pred[:,2])

In [ ]:
# Precision 
precision_score(y_test[:,0], y_pred[:,0])

In [ ]:
# Precision 
precision_score(y_test[:,1], y_pred[:,1])

In [ ]:
# Precision 
precision_score(y_test[:,2], y_pred[:,2])

## Label-1 Individual Neural network (Presence of Peacock sound)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, la1, test_size = 0.25)


#X_train,y_train = features,labels
#X_test , y_test = fea2,la2

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

print(X_train.shape)
print(X_test.shape)

#Initializing Neural Network
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 40, init = 'uniform', activation = 'relu', input_dim = 193))

# Adding the second hidden layer
classifier.add(Dense(output_dim = 20, init = 'uniform', activation = 'relu'))

classifier.add(Dense(output_dim = 8, init = 'uniform', activation = 'relu'))


# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))


# Compiling Neural Network
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting our model 
history1=classifier.fit(X_train, y_train,validation_data=(X_test, y_test), batch_size = 10, nb_epoch = 100)

#acc = []
# Predicting the Test set results
y_pred = classifier.predict(X_test)

y_pred = (y_pred > 0.5)
print('hiiiiii')
print(y_test)
print(y_pred)

err2=0
for i in range(len(y_test)):
    if((y_test[i]!=y_pred[i])):
        err2 = err2+1;
print('error')
print(err2)

scores = classifier.evaluate(x=X_test, y=y_test, batch_size=None, verbose=2, sample_weight=None, steps=None)

print('Test loss:', scores)

#acc.append(accuracy)

In [ ]:
plot_history(history1)

## Label-2 Individual Neural network (Presence of Disturbance)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, la2, test_size = 0.25)


#X_train,y_train = features,labels
#X_test , y_test = fea2,la2

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

print(X_train.shape)
print(X_test.shape)

#Initializing Neural Network
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 40, init = 'uniform', activation = 'relu', input_dim = 193))

# Adding the second hidden layer
classifier.add(Dense(output_dim = 20, init = 'uniform', activation = 'relu'))

classifier.add(Dense(output_dim = 8, init = 'uniform', activation = 'relu'))


# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))


# Compiling Neural Network
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting our model 
history2=classifier.fit(X_train, y_train,validation_data=(X_test, y_test), batch_size = 10, nb_epoch = 150)

#acc = []
# Predicting the Test set results
y_pred = classifier.predict(X_test)

y_pred = (y_pred > 0.5)
print('hiiiiii')
print(y_test)
print(y_pred)

err3=0
for i in range(len(y_test)):
    if((y_test[i]!=y_pred[i])):
        err3 = err3+1;
print('error')
print(err3)

scores = classifier.evaluate(x=X_test, y=y_test, batch_size=None, verbose=2, sample_weight=None, steps=None)

print('Test loss:', scores)

#acc.append(accuracy)

# Plots - Results

In [ ]:
plot_history(history2)

In [ ]:
# Creating the Confusion Matrix
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,cohen_kappa_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
# Precision 
precision_score(y_test, y_pred)

In [ ]:
# Recall
recall_score(y_test, y_pred)

In [ ]:
# F1 score
f1_score(y_test,y_pred)

In [ ]:
# Cohen's kappa
cohen_kappa_score(y_test, y_pred)

## Label-3 Individual Neural network (Presence of Multiple peacocks)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, la3, test_size = 0.25)


#X_train,y_train = features,labels
#X_test , y_test = fea2,la2

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

print(X_train.shape)
print(X_test.shape)

#Initializing Neural Network
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 40, init = 'uniform', activation = 'relu', input_dim = 193))

# Adding the second hidden layer
classifier.add(Dense(output_dim = 20, init = 'uniform', activation = 'relu'))

classifier.add(Dense(output_dim = 8, init = 'uniform', activation = 'relu'))


# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))


# Compiling Neural Network
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting our model 
history3=classifier.fit(X_train, y_train,validation_data=(X_test, y_test), batch_size = 10, nb_epoch = 150)

#acc = []
# Predicting the Test set results
y_pred = classifier.predict(X_test)

y_pred = (y_pred > 0.5)
print('hiiiiii')
print(y_test)
print(y_pred)

err3=0
for i in range(len(y_test)):
    if((y_test[i]!=y_pred[i])):
        err3 = err3+1;
print('error')
print(err3)

scores = classifier.evaluate(x=X_test, y=y_test, batch_size=None, verbose=2, sample_weight=None, steps=None)

print('Test loss:', scores)

#acc.append(accuracy)

In [ ]:
plot_history(history3)

In [ ]:
# Creating the Confusion Matrix
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,cohen_kappa_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
# Precision 
precision_score(y_test, y_pred)

In [ ]:
# Recall
recall_score(y_test, y_pred)

In [ ]:
# F1 score
f1_score(y_test,y_pred)